In [70]:
import os
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
from eth_account import Account

In [71]:
from pathlib import Path
from getpass import getpass

In [72]:
from constants import BTC
from constants import ETH
from constants import BTCTEST
from constants import LTC

In [73]:
load_dotenv()

True

In [74]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [75]:
account_one = Account.from_key("0x409229a3e6d2a00c67c48dc73f5ce61efb848c1faef8d4ad5009c6d300adab79")
account_one

In [76]:
type(account_one)

eth_account.signers.local.LocalAccount

In [77]:
def priv_key_to_account (coin,priv_key):
    """
    This function return the address of a wallet
    given its corresponding private key
    """
    if coin==ETH:
        account_address = Account.privateKeyToAccount(priv_key).address
    if coin==BTCTEST:
        account_address = PrivateKeyTestnet(priv_key).address
    if coin==BTC:
        account_address = PrivateKey(priv_key).address

    return account_address

In [78]:
sender_address = priv_key_to_account(ETH,"0x409229a3e6d2a00c67c48dc73f5ce61efb848c1faef8d4ad5009c6d300adab79")
sender_address

'0xef7AF838023A8710F697fFc4C5AbcbDE8610234c'

In [79]:
def create_tx(sender_acc_priv_key,recipient_address,amount,coin):
    if coin==ETH:
        my_address_to_display = priv_key_to_account(coin,sender_acc_priv_key)
        gasEstimate = w3.eth.estimateGas(
            {"from": my_address_to_display, "to": recipient_address, "value": amount}
        )
        my_gas_incentive = gasEstimate
        return {
            "from": my_address_to_display,
            "to": recipient_address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": my_gas_incentive,
            "nonce": w3.eth.getTransactionCount(my_address_to_display),
        }

In [85]:
prep_tran = create_tx("0x6ef4ae58135895c82bd476596dcbe60366b007f3ac9f3d747db840a79f538ab0",'0x454453A0732CBE6c1009342E580Fd76a4F8EF1C7',5000000, ETH)
prep_tran

{'from': '0x48566d313CDC067187Ca36E7a5991C442DF67EF0',
 'to': '0x454453A0732CBE6c1009342E580Fd76a4F8EF1C7',
 'value': 5000000,
 'gasPrice': 1000000000,
 'gas': 21000,
 'nonce': 0}

In [89]:
signer_key = Account.from_key("0x6ef4ae58135895c82bd476596dcbe60366b007f3ac9f3d747db840a79f538ab0")
signer_key

In [90]:
signed_tx = signer_key.sign_transaction(prep_tran)
signed_tx

SignedTransaction(rawTransaction=HexBytes('0xf86680843b9aca0082520894454453a0732cbe6c1009342e580fd76a4f8ef1c7834c4b40801ca07344bc5cba04936bd6f0fee455d0cea110de677a1ddb00f7f83c7ef5e9406506a00ca7051d46a843ee66235c2cb090ab9cf3f2417d9639364b23d4b61fe0bac014'), hash=HexBytes('0x875a441ebeedd492118e2d70762d59a0fb552f02e9799d72c3d40ab053093c59'), r=52137423215698823452916538716622341839262458043918361269497717703630684972294, s=5722852940828454913217883464321733570650707868219722970319674110283557290004, v=28)

In [91]:
result =w3.eth.sendRawTransaction(signed_tx.rawTransaction)
result

HexBytes('0x875a441ebeedd492118e2d70762d59a0fb552f02e9799d72c3d40ab053093c59')

In [95]:
result.hex()

'0x875a441ebeedd492118e2d70762d59a0fb552f02e9799d72c3d40ab053093c59'

In [104]:
import time

In [105]:
def send_tx(sender_acc_priv_key,recipient_address,amount,coin):
    tx = create_tx(sender_acc_priv_key,recipient_address,amount,coin)
    signer_key = Account.from_key(sender_acc_priv_key)
    signed_tx = signer_key.sign_transaction(tx)
    time.sleep(20)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()

In [106]:
send_tx("0x6ef4ae58135895c82bd476596dcbe60366b007f3ac9f3d747db840a79f538ab0",'0x454453A0732CBE6c1009342E580Fd76a4F8EF1C7',60000000, ETH)

ValueError: {'code': -32000, 'message': 'replacement transaction underpriced'}

In [107]:
signer_key2 = Account.from_key("0x6ef4ae58135895c82bd476596dcbe60366b007f3ac9f3d747db840a79f538ab0")
signer_key2

In [108]:
signed_tx2 = signer_key2.sign_transaction(prep_tran)
signed_tx2

SignedTransaction(rawTransaction=HexBytes('0xf86680843b9aca0082520894454453a0732cbe6c1009342e580fd76a4f8ef1c7834c4b40801ca07344bc5cba04936bd6f0fee455d0cea110de677a1ddb00f7f83c7ef5e9406506a00ca7051d46a843ee66235c2cb090ab9cf3f2417d9639364b23d4b61fe0bac014'), hash=HexBytes('0x875a441ebeedd492118e2d70762d59a0fb552f02e9799d72c3d40ab053093c59'), r=52137423215698823452916538716622341839262458043918361269497717703630684972294, s=5722852940828454913217883464321733570650707868219722970319674110283557290004, v=28)

In [109]:
result2 =w3.eth.sendRawTransaction(signed_tx2.rawTransaction)
result2

ValueError: {'code': -32000, 'message': 'already known'}

In [110]:
def sign_tx(sender_acc_priv_key,recipient_address,amount,coin):
    if coin==ETH:
        prep_tran = create_tx(sender_acc_priv_key,recipient_address,amount,coin)
        signed_tx = signer_key.sign_transaction(prep_tran)
        return signed_tx

In [111]:
sign_eth_tx15 = sign_tx("0x6ef4ae58135895c82bd476596dcbe60366b007f3ac9f3d747db840a79f538ab0",'0x454453A0732CBE6c1009342E580Fd76a4F8EF1C7',60000000, ETH)

In [112]:
sign_eth_tx15

SignedTransaction(rawTransaction=HexBytes('0xf86780843b9aca0082520894454453a0732cbe6c1009342e580fd76a4f8ef1c78403938700801ba0c5d54cc6f14698d1ef1e63388eb51125e7d19317f4175393f0a4f1c4bd73dda0a04cb14679ae2c0e1b856568dd018f9f96df2f6832742b046a9b84efd9cd2c42b3'), hash=HexBytes('0xd683a8f634a55536937b0d217f513d111b6884e4a6f6267b665bdd4bc5135f12'), r=89482499491902374744656192973280332529332765569314814996394876851015050845600, s=34688994825534303577746129275995277293020852335345423469318159840281319195315, v=27)

In [113]:
def send_tx_id(sender_acc_priv_key,recipient_address,amount,coin):
    if coin==ETH:
        signed_tx = sign_tx(sender_acc_priv_key,recipient_address,amount,coin)
        broadcast_tx = w3.eth.sendRawTransaction(signed_tx2.rawTransaction)
        return broadcast_tx

In [114]:
katchcoin_trans_25 = send_tx_id("0x6ef4ae58135895c82bd476596dcbe60366b007f3ac9f3d747db840a79f538ab0",'0x454453A0732CBE6c1009342E580Fd76a4F8EF1C7',700000000, ETH)

ValueError: {'code': -32000, 'message': 'already known'}